**DSCI 5260- SECTION 001
BUSINESS PROCESS ANALYTICS**

**Group 6- Project Milestone Report**

**Importing necessary libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

**Handling missing values**

In [ ]:
# Load the dataset
df = pd.read_csv('/content/TaxiTripDataDecmeber2023.csv')

# Display first five rows
print("First five rows:")
print(df.head(5))

# Show missing values per column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)

# Display missing values
print("\nMissing Values Summary:")
print(missing_df)

# Drop columns with excessive missing values (>80%)
cols_to_drop = missing_df[missing_df['Percentage'] > 80].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)
print(f"\nDropped columns due to excessive missing values: {cols_to_drop}")

# Fill missing values
df_filled = df.copy()

# Convert datetime columns
df_filled['ORIGINDATETIME_TR'] = pd.to_datetime(df_filled['ORIGINDATETIME_TR'], errors='coerce')
df_filled['DESTINATIONDATETIME_TR'] = pd.to_datetime(df_filled['DESTINATIONDATETIME_TR'], errors='coerce')

# Feature Engineering: Calculate trip duration (minutes)
df_filled['trip_duration'] = (df_filled['DESTINATIONDATETIME_TR'] - df_filled['ORIGINDATETIME_TR']).dt.total_seconds() / 60

# Fill missing numeric values
numeric_cols = df_filled.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    if df_filled[col].isnull().sum() > 0:
        if df_filled[col].skew() > 1:  # If skewed, use median
            df_filled[col].fillna(df_filled[col].median(), inplace=True)
        else:  # Otherwise, use mean
            df_filled[col].fillna(df_filled[col].mean(), inplace=True)

# Fill categorical missing values with mode
categorical_cols = df_filled.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    if df_filled[col].isnull().sum() > 0:
        df_filled[col].fillna(df_filled[col].mode()[0], inplace=True)

print("\nMissing values handled successfully!")

In [ ]:
df_filled.head(5)

**Finding unique values in ORIGINCITY, to remove unwanted characters**

In [ ]:
df_filled['ORIGINCITY'].unique()

**Removing unwanted characters**

In [ ]:
# Convert to title case and strip spaces
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].str.strip().str.title()

# Replace invalid values with NaN
invalid_values = ["???", "-", "None", "Unknown", "Na", "106", "101", "Suite #203"]
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].replace(invalid_values, np.nan)

# Define common name corrections
name_corrections = {
    "Washington Dc": "Washington",
    "Mclean": "McLean",
    "Oxon Hill Md": "Oxon Hill",
    "District Heights Md": "District Heights",
    "Falls Church Va": "Falls Church",
    "Suitland-Silver Hill": "Suitland",
    "New Carrollton Md": "New Carrollton"
}

# Apply corrections
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].replace(name_corrections)

# Remove rows with full addresses
df_filled = df_filled[~df_filled['ORIGINCITY'].str.contains(r'\d', na=False)]

# Check the cleaned unique values
print(df_filled['ORIGINCITY'].unique())

In [ ]:
print((df_filled['ORIGINCITY'] == "UNKNOWN").sum())

In [ ]:
print((df_filled['ORIGINCITY'] == "-").sum())


In [ ]:
print((df_filled['ORIGINSTATE'].str.strip() == "NA").sum())
print(df_filled['ORIGINSTATE'].isna().sum())



**Finding unique values in DESTINATIONSTATE, to remove unwanted characters**

In [ ]:
print(df_filled['DESTINATIONSTATE'].unique())

**Removing unwanted characters from DESTINATIONSTATE**

In [ ]:
import pandas as pd
import numpy as np

# List of valid U.S. state abbreviations
valid_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
    'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
    'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
}

# Strip spaces
df_filled['DESTINATIONSTATE'] = df['DESTINATIONSTATE'].str.strip()

# Replace invalid values with NaN
invalid_values = ["- ", "Na", "None", "Unknown", "", "nan"]
df_filled['DESTINATIONSTATE'] = df['DESTINATIONSTATE'].replace(invalid_values, np.nan)

# Keep only valid state abbreviations
df_filled['DESTINATIONSTATE'] = df_filled['DESTINATIONSTATE'].apply(lambda x: x if x in valid_states else np.nan)

# Check cleaned unique values
print(df_filled['DESTINATIONSTATE'].unique())

**Finding unique values in DESTINATIONCITY, to remove unwanted characters**

In [ ]:
print(df_filled['DESTINATIONCITY'].unique())

**Removing unwanted characters from DESTINATIONCITY**

In [ ]:
df_filled['DESTINATIONCITY'] = df_filled['DESTINATIONCITY'].str.strip()  # Remove leading/trailing spaces
df_filled['DESTINATIONCITY'] = df_filled['DESTINATIONCITY'].str.title()  # Standardize capitalization

# Replace unwanted values
invalid_values = ["Unknown", "-", "???", "None", "UNKNOWN"]
df_filled['DESTINATIONCITY'] = df_filled['DESTINATIONCITY'].replace(invalid_values, "Missing")

# Display unique values after cleaning
print(df_filled['DESTINATIONCITY'].unique())

In [ ]:
print((df_filled['DESTINATIONCITY']== "-").sum())

In [ ]:
print((df_filled['DESTINATIONCITY']== "UNKNOWN").sum())

In [ ]:
print((df_filled['DESTINATIONCITY']== "???").sum())

**Finding number of null values**

In [ ]:
columns = ['ORIGIN_BLOCK_LATITUDE',
            'ORIGIN_BLOCK_LONGITUDE',
            'ORIGIN_BLOCKNAME',
            'DESTINATION_BLOCK_LAT',
            'DESTINATION_BLOCK_LONG',
            'DESTINATION_BLOCKNAME',
            'AIRPORT']

for column in columns:
    print(f"Unique values in {column}:")
    print(df_filled[column].unique())
    print(df_filled[column].isna().sum())
    print()

In [ ]:
print((df_filled['ORIGIN_BLOCKNAME'] == "").sum())

**Finding unique values in DESTINATIONZIP, to remove unwanted characters**

In [ ]:
print(df_filled['DESTINATIONZIP'].unique())

**Removing every value this is not 5 numerical character length**


In [ ]:
def clean_zip(zip_code):
    zip_code = str(zip_code).strip()  # Remove leading/trailing spaces
    if zip_code.isdigit() and len(zip_code) == 5:  # Valid 5-digit ZIP
        return zip_code
    return None  # Drop invalid ZIP codes

# Apply cleaning function
df_filled['DESTINATIONZIP'] = df_filled['DESTINATIONZIP'].apply(clean_zip)

In [ ]:
print(df_filled['DESTINATIONZIP'].unique())

**Finding Unique values in ORIGINZIP to remove unwanted characters**

In [ ]:
print(df_filled['ORIGINZIP'].unique())

**Removing every value this is not 5 numerical character length**

In [ ]:
def clean_zip(zip_code):
    zip_code = str(zip_code).strip()  # Remove leading/trailing spaces
    if zip_code.isdigit() and len(zip_code) == 5:  # Valid 5-digit ZIP
        return zip_code
    return None  # Drop invalid ZIP codes

# Apply cleaning function
df_filled['ORIGINZIP'] = df_filled['ORIGINZIP'].apply(clean_zip)

In [ ]:
print(df_filled['ORIGINZIP'].unique())

In [ ]:
df_filled.head(5)

In [ ]:
print(df_filled['DESTINATIONCITY'].value_counts())


**Finding out if there is Missing as the value in DESTINATIONCITY column**

In [ ]:
df_filled.loc[df_filled['DESTINATIONCITY'] == 'Missing', ['DESTINATIONCITY']]


In [ ]:
df_filled.shape

**Removing rows with missing as the value**

In [ ]:
columns_to_check = ['DESTINATIONCITY', 'DESTINATIONZIP', 'DESTINATIONSTATE', 'ORIGINZIP', 'ORIGINSTATE']

# Remove rows with 'Missing' or empty values in the specified columns
df_filled = df_filled[~df_filled[columns_to_check].isin(['Missing', '', 'None']).any(axis=1)]

columns_to_check = ['DESTINATIONCITY', 'DESTINATIONZIP', 'DESTINATIONSTATE', 'ORIGINZIP', 'ORIGINSTATE']

# Remove rows with 'Missing' or empty values in the specified columns
df_filled = df_filled[~df_filled[columns_to_check].isin(['Missing', '']).any(axis=1)]

In [ ]:
df_filled.loc[df_filled['DESTINATIONCITY'] == 'Missing', ['DESTINATIONCITY']]

**Checking for NULL values**

In [ ]:
df_filled.loc[df_filled[columns_to_check].eq('').any(axis=1)]


In [ ]:
df_filled.loc[df_filled['AIRPORT'].eq('')]


**Checking for Unique values in AIRPORT column**

In [ ]:
df_filled['AIRPORT'].unique()


**Checking for NONE values in ORIGINZIP**

In [ ]:
df_filled['ORIGINZIP'].head(10)

In [ ]:
df_filled = df_filled.dropna(subset=['ORIGINZIP'])
df_filled = df_filled[~df_filled['ORIGINZIP'].eq('')]
df_filled['ORIGINZIP'].head(10)

**Checking for NONE values in DESTINATIONZIP**

In [ ]:
df_filled['DESTINATIONZIP'].head(10)

**Removing None values**

In [ ]:
df_filled = df_filled.dropna(subset=['DESTINATIONZIP'])
df_filled = df_filled[~df_filled['DESTINATIONZIP'].eq('')]
df_filled['ORIGINZIP'].head(10)

#**k-Nearest Neighbors (k-NN) Imputer testing**

**Finding out columns to impute using KNNImputer**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('/content/TaxiTripDataDecmeber2023.csv')

# Show missing values per column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)

print("\nMissing Values Summary:")
print(missing_df)

# Drop columns with excessive missing values (>80%)
cols_to_drop = missing_df[missing_df['Percentage'] > 80].index.tolist()
df.drop(columns=cols_to_drop, inplace=True)
print(f"\nDropped columns due to excessive missing values: {cols_to_drop}")

# Create a copy for imputation
df_filled = df.copy()

# Convert datetime columns
df_filled['ORIGINDATETIME_TR'] = pd.to_datetime(df_filled['ORIGINDATETIME_TR'], errors='coerce')
df_filled['DESTINATIONDATETIME_TR'] = pd.to_datetime(df_filled['DESTINATIONDATETIME_TR'], errors='coerce')

# Feature Engineering: Calculate trip duration (minutes)
df_filled['trip_duration'] = (df_filled['DESTINATIONDATETIME_TR'] - df_filled['ORIGINDATETIME_TR']).dt.total_seconds() / 60

# Separate numeric and categorical columns
numeric_cols = df_filled.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_filled.select_dtypes(include=['object']).columns.tolist()

# Identify numeric columns for KNN imputation (5% to 50% missing)
impute_candidates = missing_df[
    (missing_df['Percentage'] > 5) &
    (missing_df['Percentage'] <= 25) &
    (missing_df.index.isin(numeric_cols))
].index.tolist()

print(f"\nColumns selected for KNN imputation: {impute_candidates}")

**Preparing Data to pass through KNNImputer**

In [ ]:
if impute_candidates:
    # Subset the numeric columns to impute
    df_numeric = df_filled[numeric_cols]

    # Standardize the data (KNN works better with scaled data)
    scaler = StandardScaler()
    df_numeric_scaled = scaler.fit_transform(df_numeric)

    # Apply KNN Imputer
    imputer = KNNImputer(n_neighbors=2)
    df_numeric_imputed = imputer.fit_transform(df_numeric_scaled)

    # Inverse transform to original scale
    df_numeric_imputed = scaler.inverse_transform(df_numeric_imputed)

    # Update the dataframe
    df_filled[numeric_cols] = df_numeric_imputed

**Imputing rest of the columns with Mean/Median**

In [ ]:
# Impute remaining numeric columns with low missingness (<5%) using mean/median
for col in numeric_cols:
    if col not in impute_candidates and df_filled[col].isnull().sum() > 0:
        if df_filled[col].skew() > 1:  # If skewed, use median
            df_filled[col].fillna(df_filled[col].median(), inplace=True)
        else:  # Otherwise, use mean
            df_filled[col].fillna(df_filled[col].mean(), inplace=True)

# Fill categorical missing values with mode
for col in categorical_cols:
    if df_filled[col].isnull().sum() > 0:
        df_filled[col].fillna(df_filled[col].mode()[0], inplace=True)

print("\nMissing values handled successfully with KNN Imputer for selected columns!")


df_filled.head(5)

**Dowloading the cleaned Dataset**

In [ ]:
# Save the processed DataFrame to a CSV file
df_filled.to_csv('/content/TaxiTripDataDecmeber2023cleaned.csv', index=False)

In [ ]:
df_filled.shape

In [ ]:
df_filled = pd.read_csv('/content/TaxiTripDataDecmeber2023cleaned.csv')

#Removing Outliers

In [ ]:
print("Shape before removing outliers:", df_filled.shape)

**Finding Inter-quartile range**

In [ ]:
for col in ['MILEAGE', 'DURATION', 'TOTALAMOUNT', 'ORIGIN_BLOCK_LATITUDE',
    'ORIGIN_BLOCK_LONGITUDE',
    'DESTINATION_BLOCK_LAT',
    'DESTINATION_BLOCK_LONG']:
    Q1 = df_filled[col].quantile(0.25)
    Q3 = df_filled[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filled = df_filled[(df_filled[col] >= lower_bound) & (df_filled[col] <= upper_bound)]

print("Shape after removing outliers:", df_filled.shape)

In [ ]:
df_filled.to_csv('/content/TaxiTripDataDecmeber2023cleaned_outlier_removed.csv', index=False)

In [ ]:
df_filled = pd.read_csv('/content/TaxiTripDataDecmeber2023cleaned_outlier_removed.csv')

**Doing further cleaning of Data to make it more efficient**

In [ ]:
import pandas as pd
import numpy as np

# Step 1: Clean ORIGINCITY
print("\nCleaning ORIGINCITY...")
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].str.strip().str.title()

# Replace invalid values with NaN
invalid_values = ["???", "-", "None", "Unknown", "Na", "106", "101", "Suite #203", "UNKNOWN", "- "]
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].replace(invalid_values, np.nan)

# Apply name corrections
name_corrections = {
    "Washington Dc": "Washington",
    "Mclean": "McLean",
    "Oxon Hill Md": "Oxon Hill",
    "District Heights Md": "District Heights",
    "Falls Church Va": "Falls Church",
    "Suitland-Silver Hill": "Suitland",
    "New Carrollton Md": "New Carrollton"
}
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].replace(name_corrections)

# Remove rows with full addresses
df_filled = df_filled[~df_filled['ORIGINCITY'].str.contains(r'\d', na=False)]

# Fill NaN with mode
df_filled['ORIGINCITY'] = df_filled['ORIGINCITY'].fillna(df_filled['ORIGINCITY'].mode()[0])

# Check results
print("Unique values in ORIGINCITY after cleaning:")
print(df_filled['ORIGINCITY'].unique())
print("Count of '-' in ORIGINCITY:", (df_filled['ORIGINCITY'] == "-").sum())
print("Count of '???' in ORIGINCITY:", (df_filled['ORIGINCITY'] == "???").sum())

# Step 2: Clean DESTINATIONCITY
print("\nCleaning DESTINATIONCITY...")
df_filled['DESTINATIONCITY'] = df_filled['DESTINATIONCITY'].str.strip().str.title()

# Replace invalid values
invalid_values = ["Unknown", "-", "???", "None", "UNKNOWN", "- "]
df_filled['DESTINATIONCITY'] = df_filled['DESTINATIONCITY'].replace(invalid_values, "Missing")

# Check results
print("Unique values in DESTINATIONCITY after cleaning:")
print(df_filled['DESTINATIONCITY'].unique())
print("Count of '-' in DESTINATIONCITY:", (df_filled['DESTINATIONCITY'] == "-").sum())
print("Count of '???' in DESTINATIONCITY:", (df_filled['DESTINATIONCITY'] == "???").sum())
print("Count of 'UNKNOWN' in DESTINATIONCITY:", (df_filled['DESTINATIONCITY'] == "UNKNOWN").sum())

# Step 3: Clean ORIGINSTATE and DESTINATIONSTATE
print("\nCleaning ORIGINSTATE and DESTINATIONSTATE...")
valid_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
    'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
    'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
}

# Clean ORIGINSTATE
df_filled['ORIGINSTATE'] = df_filled['ORIGINSTATE'].str.strip()
invalid_values = ["- ", "Na", "None", "Unknown", "", "nan", "-"]
df_filled['ORIGINSTATE'] = df_filled['ORIGINSTATE'].replace(invalid_values, np.nan)
df_filled['ORIGINSTATE'] = df_filled['ORIGINSTATE'].apply(lambda x: x if x in valid_states else np.nan)
df_filled['ORIGINSTATE'] = df_filled['ORIGINSTATE'].fillna(df_filled['ORIGINSTATE'].mode()[0])

# Clean DESTINATIONSTATE
df_filled['DESTINATIONSTATE'] = df_filled['DESTINATIONSTATE'].str.strip()
df_filled['DESTINATIONSTATE'] = df_filled['DESTINATIONSTATE'].replace(invalid_values, np.nan)
df_filled['DESTINATIONSTATE'] = df_filled['DESTINATIONSTATE'].apply(lambda x: x if x in valid_states else np.nan)
df_filled['DESTINATIONSTATE'] = df_filled['DESTINATIONSTATE'].fillna(df_filled['DESTINATIONSTATE'].mode()[0])

# Check results
print("Unique values in ORIGINSTATE after cleaning:")
print(df_filled['ORIGINSTATE'].unique())
print("Count of '-' in ORIGINSTATE:", (df_filled['ORIGINSTATE'] == "-").sum())

print("Unique values in DESTINATIONSTATE after cleaning:")
print(df_filled['DESTINATIONSTATE'].unique())
print("Count of '-' in DESTINATIONSTATE:", (df_filled['DESTINATIONSTATE'] == "-").sum())

#Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style
sns.set(style="whitegrid")

# Ensure datetime format
df_filled['ORIGINDATETIME_TR'] = pd.to_datetime(df_filled['ORIGINDATETIME_TR'], errors='coerce')

# Basic Information
print("=== Basic Information ===")
print("\nDataset Shape:", df_filled.shape)
print("\nColumn Data Types:")
print(df_filled.dtypes)
print("\nMissing Values After Cleaning:")
print(df_filled.isnull().sum())

# Numeric & Categorical Summary
print("\n=== Summary Statistics for Numeric Columns ===")
numeric_cols = ['DURATION', 'MILEAGE', 'TOTALAMOUNT',
                'ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE',
                'DESTINATION_BLOCK_LAT', 'DESTINATION_BLOCK_LONG']
numeric_cols = [col for col in numeric_cols if col in df_filled.columns]
print(df_filled[numeric_cols].describe())

print("\n=== Summary for Categorical Columns ===")
categorical_cols = ['ORIGINCITY', 'DESTINATIONCITY', 'ORIGINSTATE', 'DESTINATIONSTATE', 'AIRPORT']
categorical_cols = [col for col in categorical_cols if col in df_filled.columns]
for col in categorical_cols:
    print(f"\nValue Counts for {col}:")
    print(df_filled[col].value_counts().head(10))

# Create plot directory
if not os.path.exists('eda_plots'):
    os.makedirs('eda_plots')

# Histograms
print("\nGenerating Histograms...")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 3, i)
    sns.histplot(df_filled[col], bins=30, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    if i >= 9:
        break
plt.tight_layout()
plt.show()

# Boxplots
print("\nGenerating Boxplots...")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=df_filled[col])
    plt.title(f'Boxplot of {col}')
    if i >= 9:
        break
plt.tight_layout()
plt.show()

# Correlation Matrix
print("\nGenerating Correlation Matrix...")
plt.figure(figsize=(10, 8))
corr_matrix = df_filled[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numeric Features')
plt.show()

# Remove outliers
print("Shape before removing outliers:", df_filled.shape)

# IQR for regular numeric columns (excluding trip_duration)
for col in ['MILEAGE', 'DURATION', 'TOTALAMOUNT',
            'ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE',
            'DESTINATION_BLOCK_LAT', 'DESTINATION_BLOCK_LONG']:
    Q1 = df_filled[col].quantile(0.25)
    Q3 = df_filled[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filled = df_filled[(df_filled[col] >= lower_bound) & (df_filled[col] <= upper_bound)]

# Custom rule for trip_duration (1 to 1440 minutes)
if 'trip_duration' in df_filled.columns:
    df_filled = df_filled[(df_filled['trip_duration'] > 1) & (df_filled['trip_duration'] < 1440)]

print("Shape after removing outliers:", df_filled.shape)

# Scatter plot: trip_duration vs TOTALAMOUNT
print("\nGenerating Scatter Plots...")
if 'trip_duration' in df_filled.columns and 'TOTALAMOUNT' in df_filled.columns:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='trip_duration', y='TOTALAMOUNT', data=df_filled)
    plt.title('Trip Duration vs Total Amount')
    plt.xlabel('Trip Duration (minutes)')
    plt.ylabel('Total Amount ($)')
    plt.show()

# Scatter plot: MILEAGE vs TOTALAMOUNT
if 'MILEAGE' in df_filled.columns and 'TOTALAMOUNT' in df_filled.columns:
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='MILEAGE', y='TOTALAMOUNT', data=df_filled)
    plt.title('Mileage vs Total Amount')
    plt.xlabel('Mileage (miles)')
    plt.ylabel('Total Amount ($)')
    plt.show()

# Temporal Analysis
if 'ORIGINDATETIME_TR' in df_filled.columns:
    print("\nGenerating Temporal Analysis...")
    if df_filled['ORIGINDATETIME_TR'].dtype != 'datetime64[ns]':
        df_filled['ORIGINDATETIME_TR'] = pd.to_datetime(df_filled['ORIGINDATETIME_TR'], errors='coerce')

    # Drop invalid dates
    initial_shape = df_filled.shape
    df_filled = df_filled.dropna(subset=['ORIGINDATETIME_TR'])
    print(f"Dropped {initial_shape[0] - df_filled.shape[0]} rows with NaT in ORIGINDATETIME_TR")

    # Extract hour/day
    df_filled['hour_of_day'] = df_filled['ORIGINDATETIME_TR'].dt.hour
    df_filled['day_of_week'] = df_filled['ORIGINDATETIME_TR'].dt.day_name()

    # Trips by Hour
    plt.figure(figsize=(10, 6))
    sns.countplot(x='hour_of_day', data=df_filled)
    plt.title('Number of Trips by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Trips')
    plt.show()

    # Trips by Day
    plt.figure(figsize=(10, 6))
    sns.countplot(x='day_of_week', data=df_filled, order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    plt.title('Number of Trips by Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Number of Trips')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("\nORIGINDATETIME_TR not found in DataFrame. Skipping temporal analysis.")




In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

df_filled = df_filled.dropna(subset=['ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE'])

map_center = [df_filled['ORIGIN_BLOCK_LATITUDE'].mean(), df_filled['ORIGIN_BLOCK_LONGITUDE'].mean()]
m = folium.Map(location=map_center, zoom_start=11)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in df_filled.iterrows():
    folium.CircleMarker(
        location=[row['ORIGIN_BLOCK_LATITUDE'], row['ORIGIN_BLOCK_LONGITUDE']],
        radius=4,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=f"Origin: ({row['ORIGIN_BLOCK_LATITUDE']:.5f}, {row['ORIGIN_BLOCK_LONGITUDE']:.5f})"
    ).add_to(marker_cluster)

# Show the map
m


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

df_filled = df_filled.dropna(subset=['DESTINATION_BLOCK_LAT', 'DESTINATION_BLOCK_LONG'])

map_center = [df_filled['DESTINATION_BLOCK_LAT'].mean(), df_filled['DESTINATION_BLOCK_LONG'].mean()]
m = folium.Map(location=map_center, zoom_start=11)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in df_filled.iterrows():
    folium.CircleMarker(
        location=[row['DESTINATION_BLOCK_LAT'], row['DESTINATION_BLOCK_LONG']],
        radius=4,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        popup=f"Destination: ({row['DESTINATION_BLOCK_LAT']:.5f}, {row['DESTINATION_BLOCK_LONG']:.5f})"
    ).add_to(marker_cluster)

# Show the map
m

In [ ]:
df_filled.columns

# **Machine Learning**

# 1. What are the most determining variables for the total fare of rides?

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

numeric_cols = df_filled.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Removing target from features list if present
target = 'TOTALAMOUNT'
if target in numeric_cols:
    numeric_cols.remove(target)

# Prepare data
X = df_filled[numeric_cols]
y = df_filled[target]

# Drop missing values
df_model = pd.concat([X, y], axis=1).dropna()
X = df_model[numeric_cols]
y = df_model[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Feature importances
importances = rf.feature_importances_

# Create a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Take Top 5 Features
top_features = feature_importance_df.head(5)

# Plot horizontally
plt.figure(figsize=(8,6))
sns.barplot(
    y='Importance',
    x='Feature',
    data=top_features,
    palette='viridis'
)
plt.title('Top 5 Feature Importance for Predicting Total Fare (Random Forest)', fontsize=14)
plt.ylabel('Importance', fontsize=12)
plt.xlabel('Feature', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.show()

# Print the top 5 feature importance table
print(top_features)


# 2. How does the ride fare change with different cities, states, and times?

 **By Origin City**

In [ ]:
# Average Fare by Origin City
city_avg = df_filled.groupby('ORIGINCITY')['TOTALAMOUNT'].mean().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
city_avg.plot(kind='bar')
plt.title('Average Fare by Top 10 Origin Cities')
plt.ylabel('Average Total Fare ($)')
plt.xticks(rotation=45)
plt.show()

**by Origin State**

In [ ]:
state_avg = df_filled.groupby('ORIGINSTATE')['TOTALAMOUNT'].mean().sort_values(ascending=False)

plt.figure(figsize=(10,6))
state_avg.plot(kind='bar')
plt.title('Average Fare by Origin States')
plt.ylabel('Average Total Fare ($)')
plt.xticks(rotation=45)
plt.show()

**by Hour of Day**

In [ ]:
# Fare by Hour of Day
hourly_avg = df_filled.groupby('hour_of_day')['TOTALAMOUNT'].mean()

plt.figure(figsize=(10,6))
hourly_avg.plot(kind='line', marker='o')
plt.title('Average Fare by Hour of Day')
plt.xlabel('Pickup Hour')
plt.ylabel('Average Total Fare ($)')
plt.grid(True)
plt.show()

**by Day of Week**

In [ ]:
# Fare by Day of Week
dow_avg = df_filled.groupby('day_of_week')['TOTALAMOUNT'].mean()

plt.figure(figsize=(10,6))
dow_avg.plot(kind='line', marker='s')
plt.title('Average Fare by Day of Week')
plt.xlabel('Day of Week (0=Monday)')
plt.ylabel('Average Total Fare ($)')
plt.grid(True)
plt.show()

# How does trip duration affect the total fare, and is it linear across time?



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np

df_plot = df_filled[df_filled['trip_duration'] <= 500]

# Scatter plot
plt.figure(figsize=(12, 7))
sns.scatterplot(x='trip_duration', y='TOTALAMOUNT', data=df_plot, alpha=0.3)

# LOWESS smoothing
lowess = sm.nonparametric.lowess
z = lowess(df_plot['TOTALAMOUNT'], df_plot['trip_duration'], frac=0.2)  # frac controls smoothing
plt.plot(z[:, 0], z[:, 1], color='red', label='LOWESS Smoothed Trend', lw=2)

# Correlation
corr = df_plot['trip_duration'].corr(df_plot['TOTALAMOUNT'])

# Titles and labels
plt.title(f'Trip Duration vs Total Fare (Smoothed Trend)\nCorrelation: {corr:.2f}', fontsize=16, fontweight='bold')
plt.xlabel('Trip Duration (minutes)', fontsize=14)
plt.ylabel('Total Fare ($)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()


# What are the most frequent payment methods, and is there any relationship with the amount of gratuity?

In [ ]:
# Mapping based on Source Data Readme file

payment_type_mapping = {
    1.0: 'Credit',
    2.0: 'Cash',
    3.0: 'EHail',
    4.0: 'Other',
    6.0: 'Uber Credit'
}
# Apply payment type labels
df_filled['PAYMENTTYPE_LABEL'] = df_filled['PAYMENTTYPE'].map(payment_type_mapping)

# Checking payment type frequency
payment_counts = df_filled['PAYMENTTYPE_LABEL'].value_counts()
print("Payment Type Counts:\n", payment_counts)

# Bar plot of payment type frequency
plt.figure(figsize=(10,6))
sns.countplot(data=df_filled, x='PAYMENTTYPE_LABEL', order=payment_counts.index, palette='viridis') # Changed 'PAYMENT_TYPE' to 'PAYMENTTYPE'
plt.title('Frequency of Payment Methods', fontsize=16)
plt.xlabel('Payment Type', fontsize=14)
plt.ylabel('Number of Rides', fontsize=14)
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.show()


# Summary statistics
gratuity_summary = df_filled.groupby('PAYMENTTYPE_LABEL')['GRATUITYAMOUNT'].describe() # Changed 'PAYMENT_TYPE' to 'PAYMENTTYPE'
print("\nGratuity Summary by Payment Type:\n", gratuity_summary)

# Predict TOTALAMOUNT based on features like MILEAGE, DURATION, FAREAMOUNT, GRATUITYAMOUNT, etc?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Features and Target
features = ['MILEAGE', 'DURATION', 'FAREAMOUNT', 'GRATUITYAMOUNT', 'trip_duration']
X = df_filled[features]
y = df_filled['TOTALAMOUNT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Custom Regression Accuracy
def regression_accuracy(y_true, y_pred, tolerance=0.10):
    within_tolerance = np.abs(y_true - y_pred) <= (tolerance * np.abs(y_true))
    return np.mean(within_tolerance) * 100

# Model Training and Evaluation Function
def train_and_evaluate(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n{model_name} Results:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
    print(f"R² Score: {r2_score(y_test, y_pred):.2f}")
    print(f"Accuracy (within 10%): {regression_accuracy(y_test, y_pred):.2f}%")

# Random Forest
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
train_and_evaluate(rf, "Random Forest")

# Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, random_state=42)
train_and_evaluate(gb, "Gradient Boosting")
